In [8]:
from PIL import Image
import numpy as np
from openvino.inference_engine import IECore

# Path to the IR files
model_xml = 'model.xml'
model_bin = 'model.bin'

# Load the Inference Engine
ie = IECore()

pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds = [0.229, 0.224, 0.225]

# Load the network
net = ie.read_network(model=model_xml, weights=model_bin)

# Get the input and output names
input_name = next(iter(net.input_info))
output_name = next(iter(net.outputs))

# Read and preprocess the input image
# Load and preprocess the image
image_path = "test.jpg"
image = Image.open(image_path).convert("RGB")
image = image.resize((224, 224))  # Resize the image to match the model's input size
image_array = np.array(image)
image_array = (image_array / 255.0 - pretrained_means) / pretrained_stds
# image_array /= 255.0  # Normalize pixel values to the range [0, 1]
image_array = np.transpose(image_array, (2, 0, 1))  # Change the shape to (3, 224, 224)
image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

# Load the model to the device
exec_net = ie.load_network(network=net, device_name='CPU')

# Perform inference
result = exec_net.infer(inputs={input_name: image_array})

# Process the output
names = []
# classes.reverse()
output_data = result['output']
# Add your post-processing logic here
print(round(np.max(output_data) * 100, 2))
print("Inference result:", names[np.argmax(output_data)])

1122.66
Inference result: back_of_right_hand
